### Isolation Forest

Данный подход не требует нормальности данных (в смысле рабочего состояния системы), т.е. в данных могут быть аномалии. **Из-за своей склонности к изоляции аномалии изолируются ближе к корню дерева.** В то время как нормальные состояния системы находится как можно глубже. При моделировании строится ансамбль деревьев для предлагаемого датасета, тогда аномалии - это те экземпляры, которые имеют короткую среднюю длину пути. Существует два гиперпараметра модели: количество деревьев в лесу и размер подвыборки.

Изолировать - это значит отделить один объект от остальных. Аномалии наиболее предрасположены к изоляции, поскольку их мало и они отличаются от нормальных объектов. В случайном дереве разбиение экземпляров рекурсивно повторяется до тех пор, пока все экземпляры не будут изолированы. Если лес случайных деревьев вместе создает более короткие пути для некоторых конкретных точек, то они, скорее всего, будут аномалиями. 

Нормальный объект требует больших разбиений для того чтобы быть изолированным, в то время как аномальный объект будет отделен на более ранней стадии. **При разбиении рандомно выбирается признак и сплит-значение** между минимальным и максимальным значениями признака.

Поскольку рекурсивное разбиение может быть представлено в деревовидной структуре, то количество разбиений соответсвует длине пути из корня в терминальную вершину. Поскольку каждое разбиение рандомно, то в отдельности деревья сгенерируют различные разбиения. Тогда для каждой вершины посмотрим его высоту в дереве и усредним по рамеру леса. При увеличении количества деревьев средняя высота будет сходиться к конкретному числу.

<img src="iforest.png" alt="Drawing" style="width: 400px; height: 600px"/>

На картинках видно, что аномалия довольно быстро отсекается, и чем больше деревьев в ансамбле тем конкретнее длина пути

**Определение Isolation Tree**. Пусть $T$ это произвольная вершина. Тогда $T$ это либо внешняя вершина без детей, либо внутрення с двумя детьми $(T_l, T_r)$ и *тестом*. Тест содержит значения признака $q$ и сплит-значение $p$, и по критерию $q < p$ разделяет данные в $T_l$ и $T_r$ соответственно.

Пусть дана выборка $X = \{x_1, x_2, \ldots, x_n\}$, тогда для построения модели будем рекурсивно разделять выборку по рандомному признаку $q$ и сплит-значению $p$ до тех пор пока:
* не будет достигнут лимит по высоте дерева (как гиперпараметр модели)
* размер выборки не будет равняться единице $|X| = 1$
* все объекты в выборке не будут иметь одинаковые значения

Isolation Tree это бинарное дерево, т.е. каждая вершина имеет не больше двух дочерних вершин. Чтобы обнаружить аномалии нужно отсортировать вершины по длине пути или *anomaly score* в порядке возрастания, тогда вверху списка будут аномальные вершины.

**Длина пути и anomaly score**. Длина пути $h(x)$ это просто количество ребер от корня дерева до вершины $x$.

Поскольку $ITree$ имеет аналогичную структуру, что и $BST$ (двоичное дерево поиска), то средняя длина внешнего пути оценивается как [неудачный поиск в BST](https://book.huihoo.com/data-structures-and-algorithms-with-object-oriented-design-patterns-in-java/html/page308.html), а здесь [более подробно про связь с гармоническими числами](https://www.cs.csustan.edu/~john/classes/previous_semesters/cs3100_datastructures/2000_04_Fall/Examples/Trees/averageSearchInBST.html). Откуда можно получить среднюю глубину внешней вершины
$$\large c(n) = 2H(n-1) - \frac{2(n-1)}{n}$$

Поскольку $c(n)$ это среднее значение, то можно использовать его для нормализации $h(x)$. Таким образом *anomaly score* для объекта $x$ определяется как $$\Large s(x, n) = 2^{-\frac{E(h(x))}{c(n)}}$$

где $E(h(x))$ это средняя длина пути для объекта $x$ на основе всех деревьев

<img src="anomaly_score.png" alt="Drawing" style="width: 400px; height: 300px"/>

Из графика видно, чем чем больше $E(h(x))$ относительно $c(n)$ тем ближе *anomaly score* к нулю и тем менее вероятно что объект $x$ является аномалией. И наоборот. Поэтому можно сделать несколько важных выводов:

* если значение близко к единице, то это определенно аномалия
* если объекты имеют значение $< 0.5$, то их можно рассматривать как нормальные

Как ансамбль деревьев, в котором используются изолированные деревья, IForest а) идентифицирует аномалии как точки с более короткой длиной пути и б) имеет несколько деревьев, действующих как «эксперты» для выявления различных аномалий. Поскольку IForest не нужно изолировать все нормальные экземпляры — большую часть обучающей выборки IForest может хорошо работать с частичной моделью, не изолируя все нормальные точки, и строит модели, используя небольшой размер выборки. При этом большой размер выборки снижает способность IForest изолировать аномалии, поскольку обычные экземпляры могут мешать процессу изоляции и, следовательно, уменьшают его способность однозначно изолировать аномалии. 

**Основые проблемы при поиске аномалий**: *swamping* и *masking*. 

*Swamping* относится к ошибочной идентификации нормальных экземпляров как аномалий (оно же ложноположительное обнаружение). Когда нормальные экземпляры находятся слишком близко к аномалиям, количество разделов, необходимых для разделения аномалий, увеличивается, что затрудняет отличить аномалии от нормальных экземпляров. 

*Masking* это наличие слишком большого количества аномалий (групповые аномалии), скрывающих свое присутствие. Когда кластер аномалий большой и плотный, также увеличивается количество разделов для изоляции каждой аномалии. 

В этих обстоятельствах оценки с использованием $Itrees$ имеют большую длину пути, что и затрудняет обнаружение аномалий. Обе проблемы вытекают из слишком большого размера выборки. Способность $IForest$ работать с частичной моделью позволяет избежать этих проблем, и этим нужно пользоваться. Это связано с тем, что: 1) подвыборка контролирует размер данных, что помогает IForest лучше изолировать аномалии и 2) каждое изолированное дерево может быть специализировано, поскольку каждая подвыборка включает различный набор аномалий или даже не содержит аномалий. 

###  Обнаружение аномалий с помощью IForest

Алгоритм состоит из двух стадий, как в классическом машинном обучении. Первая стадия - тренировка, строит $ITrees$ на подвыборках из тренировочных данных. На второй стадии - тестирование, тестовые экземпляры проходят через деревья изоляции, чтобы получить оценку аномалии для каждого экземпляра.

In [216]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [217]:
def mar(test, pred):
    p = pred.values.astype(bool)
    t = test.values.astype(bool)
    fn = (~p & t).sum()
    tp = (p & t).sum()
    return fn / (tp + fn)

def far(test, pred):
    p=pred.values.astype(bool)
    t=test.values.astype(bool)
    fp = (p & ~t).sum()
    tn = (~p & ~t).sum()
    return fp / (fp + tn)

In [218]:
def f1_score(test, pred):
    p=pred.values.astype(bool)
    t=test.values.astype(bool)
    tp = (p & t).sum()
    fp = (p & ~t).sum()
    fn = (~p & t).sum()
    return tp / (tp + 0.5 * (fp + fn))

In [219]:
import os

all_files = []
for root, dirs, files in os.walk("data/"):
    for file in files:
        f_name = f'{root}/{file}'
        if file.endswith(".csv") and "anomaly-free" not in file and "other/2.csv" not in f_name:
             all_files.append(f_name)    #other/2.csv содержит только 100 здоровых сэмплов
                

In [220]:
list_of_df = [pd.read_csv(file, 
                          sep=';', 
                          index_col='datetime', 
                          parse_dates=True) for file in all_files][13:]

In [221]:
health_prefix_lens=[]
for df in list_of_df:
    health_beginning_length = df[df.anomaly.cumsum()==0].shape[0]
    health_prefix_lens.append(health_beginning_length)
train_prefix_len = min(health_prefix_lens)
train_prefix_len # число сэмплов для обучения в начале каждого ряда

560

In [222]:
df = pd.read_csv("data/valve1/3.csv", sep=';', index_col='datetime', parse_dates=True)
df = df[health_beginning_length:]
y_test = df["anomaly"]
df.drop(["anomaly", "changepoint"], inplace=True, axis=1)
df.sort_index(inplace=True)
x_test = df

In [223]:
from sklearn.ensemble import IsolationForest


# Fitting

In [224]:
from tqdm import tqdm

In [ ]:
clf = IsolationForest(contamination=0.01, n_jobs=-1)
list_of_predictions=[]
for i, df in tqdm(enumerate(list_of_df)):
    df = df.drop(["anomaly", "changepoint"], axis=1)
    x = df.values
    clf.fit(x[:train_prefix_len])
    predictions_raw = clf.predict(x_test.values)
    predictions = pd.Series(predictions_raw).map({-1: 1, 1:0})
    list_of_predictions.append(predictions)
all_predictions = pd.DataFrame(list_of_predictions).T
voting_res = (all_predictions.mean(axis=1)>0.5).astype(int)
f1_score(y_test, voting_res), mar(y_test, voting_res), far(y_test, voting_res), all_predictions.sum(axis=0)

19it [00:05,  3.43it/s]

## Results Interpretation 

In [ ]:
health = clf.decision_function(x)
df_health = df.copy()
df_health["Health"] = health

In [ ]:
df_health.head()

In [ ]:
plt.plot(health)
plt.show()

In [ ]:
fig, ax = plt.subplots(figsize=(10, 5))

ax.set_title("Health")
ax.plot(df_health["Health"].resample("15S").mean())
ax.axhspan(-0.02, 0.02, alpha=0.2, color="red")
plt.show()

$$\large MAR = \frac{FN}{TP + FN}$$

$$\large FAR = \frac{FP}{FP + TN}$$

In [ ]:
y_test.astype(int)
# 1 аномалия есть. Positive ~ is anomaly
# 0 аномалии нет. Negative

In [ ]:
mar_res = mar(y_test, voting_res) # пропущенные

In [ ]:
far_res = far(y_test, voting_res) # ложноопределенные

$$\large F1 = \frac{TP}{TP + 0.5(FP + FN)}$$

In [ ]:
f1_res = f1_score(y_test, voting_res)

In [ ]:
# plt.subplot(2, 1, 1)
fig, axs = plt.subplots(2, 1, figsize=(20,10))
temperatures = df.Temperature
axs[0].plot(temperatures)

y = pd.Series(voting_res)
anomalies_indecies = y.index[y == 1]
axs[0].scatter(df.index[anomalies_indecies], temperatures[anomalies_indecies], color='r')
# #axs.scatter(cpi, temperatures[cpi], color='g', marker='^', s=300)
axs[0].set_title("Temperature Predicted")

axs[1].plot(temperatures)
y = pd.Series(y_test)
anomalies_indecies = pd.Series(y.values).index[y == 1]
axs[1].scatter(df.index[anomalies_indecies], temperatures[anomalies_indecies], color='r')
# #axs.scatter(cpi, temperatures[cpi], color='g', marker='^', s=300)
axs[1].set_title("Temperature")

In [ ]:
# plt.subplot(2, 1, 1)
fig, axs = plt.subplots(2, 1, figsize=(20,10))
temperatures = df.Temperature
# axs[0].plot(temperatures)

y = pd.Series(predictions)
axs[0].plot(range(len(y)), y, marker='o')
axs[0].plot(range(len(y_test)), y_test)
# axs[0].scatter(df.index[anomalies_indecies], temperatures[anomalies_indecies], color='r')
# #axs.scatter(cpi, temperatures[cpi], color='g', marker='^', s=300)
# axs[0].set_title("Temperature Predicted")

y = pd.Series(predictions)
axs[1].plot(range(len(y[:400])), y[:400], marker='o')
axs[1].plot(range(len(y_test[:400])), y_test[:400])

In [ ]:
f1_res, far_res, mar_res 

## Вывод

Если в выборке большая доля аномалий, лучше обучаться только на здоровых данных.
Обучили несколько IForest'ов на разных DF. Спрогнозировали на одном и провели голосование.

Алгоритм  |  F1  | FAR | MAR | 
--- | --- | --- |--- |
Perfect | 1 | 0 | 0
SKAB IForest | 0.4 | 0.06 | 0.72
IForest | 0.75 | 0.18 | 0.31